In [1]:
#importing numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
#importing keras and keras dataset(Mnist)
import keras 
from  keras.datasets import mnist
#sklearn imports
#sklearn for preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
#imports for the creating models
from keras.models import Sequential
from keras.layers import Dense , Conv2D , MaxPool2D ,Flatten,Dropout
from keras.callbacks import ModelCheckpoint , EarlyStopping
#importing cv2
import cv2

In [2]:
#importing datasets
(X_train , y_train) , (X_test , y_test) = mnist.load_data()


def dataset_disc(n):
    #dataset viszualisation 
    #shape of the dataset
    X_train.shape , X_test.shape
    #dataset 
    plt.title(f'The number is : {y_train[n]}')
    plt.imshow(X_train[n] , cmap='binary')
    plt.axis('off')
    plt.show()

#preprocessing data
#normalization and reshaping using Standard scalar from sklearn
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print(X_train[33])
plt.imshow(X_train[33] , cmap='binary')

In [3]:
#preprocessing data
#normalization and reshaping
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255
#expanding the dimention from (28,28) to (28,28,1)
X_train = np.expand_dims(X_train , -1)
X_test = np.expand_dims(X_test , -1)
print(X_train[1].shape)

#OneHotEncoding the y datas
y_train =  keras.utils.to_categorical(y_train)
y_test =  keras.utils.to_categorical(y_test)

(28, 28, 1)


In [4]:
#building models
model = Sequential()

model.add(Conv2D(32,input_shape = (28,28,1),kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

In [5]:
#model summary and complilation 
model.summary()
#model uses adam as optimizer and categorical_crossentropy for losses.
model.compile(optimizer='adam',loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dropout (Dropout)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 10)                54090     
                                                                 
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
____________________________________________________

In [6]:
#callbacks for the model (earlystopping and modelcheckpoint)
earlystopping = EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=4,verbose=1)
modelcheckpoint = ModelCheckpoint('./digitRecogonitionModel.h5',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
callbacks = [earlystopping,modelcheckpoint]

In [7]:
#training model
his_train =model.fit(X_train,y_train,epochs=50,validation_split=0.3,callbacks=callbacks,)

Epoch 1/50
1310/1313 [============================>.] - ETA: 0s - loss: 0.2787 - accuracy: 0.9203
Epoch 1: val_accuracy improved from -inf to 0.96422, saving model to .\digitRecogonitionModel.h5
1313/1313 [==============================] - 10s 7ms/step - loss: 0.2784 - accuracy: 0.9204 - val_loss: 0.1233 - val_accuracy: 0.9642
Epoch 2/50
1312/1313 [============================>.] - ETA: 0s - loss: 0.1034 - accuracy: 0.9696
Epoch 2: val_accuracy improved from 0.96422 to 0.97283, saving model to .\digitRecogonitionModel.h5
1313/1313 [==============================] - 10s 7ms/step - loss: 0.1034 - accuracy: 0.9696 - val_loss: 0.0889 - val_accuracy: 0.9728
Epoch 3/50
1311/1313 [============================>.] - ETA: 0s - loss: 0.0794 - accuracy: 0.9768
Epoch 3: val_accuracy improved from 0.97283 to 0.97467, saving model to .\digitRecogonitionModel.h5
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0796 - accuracy: 0.9767 - val_loss: 0.0834 - val_accuracy: 0.9747
Epoch 4

In [10]:
#loading saved model from the directory and calculating the model accuracy 
saved_model = keras.models.load_model("D:\PROGRAMMING\deepLearning\handwrittenDigitRecognition\digitRecogonitionModel.h5")
saved_model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0544 - accuracy: 0.9821


[0.054399531334638596, 0.9821000099182129]

In [28]:
#input from user 
input_image = cv2.imread('./image_3.PNG')
print(input_image.shape)
#grayscalig to reduce the dimention from 3 to 1
input_image = cv2.cvtColor(input_image,cv2.COLOR_RGB2GRAY)
print(input_image.shape)
input_image = cv2.resize(input_image,(28,28))
print(input_image.shape)
input_image = input_image/255
input_image = np.reshape(input_image , [1,28,28,1])
print(input_image.shape)

(90, 88, 3)
(90, 88)
(28, 28)
(1, 28, 28, 1)


In [29]:
#predicting using user input
predict = model.predict(input_image)
predict = np.argmax(predict)
print(predict)

1/1 [==============================] - 0s 13ms/step
3
